# Chromia

[Chromia](https://learn.chromia.com/courses/vector-db-movie-demo/introduction) is a AI-native decentralized vector database focused on developer productivity and happiness. 

This guide provides a quick overview for getting started with Chromia [`vector stores`](/docs/concepts/#vectorstores). For detailed documentation of all `Chromia` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_vectorstores_chromia.Chromia.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/vectorstores/chroma/) | Package latest |
| :--- | :--- | :---: | :---: |
| [`Chromia`](https://api.js.langchain.com/classes/langchain_community_vectorstores_chromia.Chromia.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## Setup

To use Chromia vector stores, you'll need to install the `@langchain/community` integration package along with the [Postchain JS SDK](https://www.npmjs.com/package/postchain-client) as a peer dependency.

This guide will also use [OpenAI embeddings](/docs/integrations/text_embedding/openai), which require you to install the `@langchain/openai` integration package. You can also use [other supported embeddings models](/docs/integrations/text_embedding) if you wish.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai @langchain/core postchain-client
</Npm2Yarn>
```

Next, follow the following instructions to run Chromia with Docker on your computer:

```
docker run --rm -it -p 7740:7740 registry.gitlab.com/chromaway/example-projects/directory1-example/managed-single:latest
pmc blockchain add -bc build/vector_example.xml -c dapp -n vector_blockchain
```

You can see alternative setup instructions [in this guide](https://github.com/liho00/vector-db-chromia).

### Environment variables

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
process.env.NODE_URL_POOL = "http://localhost:7740";
process.env.DIRECTORY_CHAIN_RID = "YOUR_DIRECTORY_CHAIN_RID";
process.env.BLOCKCHAIN_RID = "YOUR_BLOCKCHAIN_RID";
process.env.MERKLE_HASH_VERSION = "YOUR_MERKLE_HASH_VERSION";
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## Instantiation

In [ ]:
import { Chromia } from "@langchain/community/vectorstores/chromia";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const nodeUrlPool = process.env.NODE_URL_POOL;
if (!nodeUrlPool) throw new Error(`Expected env var NODE_URL_POOL`);

const merkleHashVersion = process.env.MERKLE_HASH_VERSION;
if (!merkleHashVersion) throw new Error(`Expected env var MERKLE_HASH_VERSION`);

const directoryChainRid = process.env.DIRECTORY_CHAIN_RID;
if (!directoryChainRid) throw new Error(`Expected env var DIRECTORY_CHAIN_RID`);

const blockchainRid = process.env.BLOCKCHAIN_RID;
if (!blockchainRid) throw new Error(`Expected env var BLOCKCHAIN_RID`);

const postchainClient = await createClient({
  nodeUrlPool: nodeUrlPool,
  merkleHashVersion: 1,
  directoryChainRid: directoryChainRid,
  blockchainRid: blockchainRid,
})

const vectorStore = new Chromia(embeddings, {
  client: postchainClient,
  numDimensions: embeddings.dimensions,
});

## Manage vector store

### Add items to vector store

In [8]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

[ '1', '2', '3', '4' ]


### Delete items from vector store

You can delete documents from Chroma by id as follows:

In [10]:
await vectorStore.delete({ ids: ["4"] });

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent. 

### Query directly

Performing a simple similarity search can be done as follows:

In [11]:
const filter = { source: "https://example.com" };

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


See [this page](https://docs.trychroma.com/guides#filtering-by-metadata) for more on Chromia filter syntax.

If you want to execute a similarity search and receive the corresponding scores you can run:

In [12]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.835] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.852] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### Query by turning into retriever

You can also transform the vector store into a [retriever](/docs/concepts/retrievers) for easier usage in your chains. 

In [13]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials: working with external knowledge](/docs/tutorials/#working-with-external-knowledge).
- [How-to: Question and answer with RAG](/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](/docs/concepts/retrieval)

## API reference

For detailed documentation of all `Chromia` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_vectorstores_chromia.Chromia.html)